# Data exploration from non-defaulting and non-defaulting customers

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv', na_values='na')

In [4]:
df.head(n=10)

# Struct

In [9]:
df.shape #(total_rows,total_columns)

In [11]:
df[df['default'] == 0].shape #Compliant

In [12]:
df[df['default'] == 1].shape #Non-Compliant

In [13]:
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape

In [15]:
print(f"Compliant customers percentage {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"Non-compliant customers percentage {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

In [16]:
df.head(n=5)

In [17]:
df.dtypes

In [18]:
df.select_dtypes('object').describe().transpose()

In [19]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

# How to treat empty data ?

Can be Empty, Null or 'na' in this case

In [22]:
df.isna().any()

In [24]:
def stats_dados_faltantes(df: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df.columns:
    if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)

In [25]:
stats_dados_faltantes(df=df)

In [26]:
stats_dados_faltantes(df=df[df['default'] == 0])

In [27]:
stats_dados_faltantes(df=df[df['default'] == 1])

# Replacing obj for float

In [28]:
df[['limite_credito', 'valor_transacoes_12m']].dtypes

In [29]:
df[['limite_credito', 'valor_transacoes_12m']].head(n=5)

Replacing obj for float and testing with map method

In [30]:
fn = lambda valor: float(valor.replace(".", "").replace(",", "."))

valores_originais = ['12.691,51', '8.256,96', '3.418,56', '3.313,03', '4.716,22']
valores_limpos = list(map(fn, valores_originais))

print(valores_originais)
print(valores_limpos)

# Aplying

In [31]:
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(fn)
df['limite_credito'] = df['limite_credito'].apply(fn)

In [32]:
df.dtypes

In [33]:
df.select_dtypes('object').describe().transpose()

In [34]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

Replacing missing data

In [36]:
df.dropna(inplace=True)

In [38]:
df.shape

we lost so many rows but, we've got more accurate data

In [39]:
df[df['default'] == 0].shape

In [40]:
df[df['default'] == 1].shape

In [42]:
qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape

In [46]:
print(f"Compliant customers percentage was {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"New Compliant customers percentage {round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)}%")
print("")
print(f"Non-compliant customers percentage was {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"New non-compliant customers percentage {round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)}%")

# Visualization with plots

In [47]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

In [48]:
df_adimplente = df[df['default'] == 0]
df_inadimplente = df[df['default'] == 1]

In [49]:
df.select_dtypes('object').head(n=5)

In [55]:
coluna = 'escolaridade'
titulos = ['Customers Education', 'Compliant Customers Education', 'Non-compliant Customers Education']

eixo = 0
max_y = 0
max = df.select_dtypes('object').describe()[coluna]['freq'] * 1.1

figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  df_to_plot = dataframe[coluna].value_counts().to_frame()
  df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
  df_to_plot[coluna] = df_to_plot.index
  df_to_plot.sort_values(by=[coluna], inplace=True)
  df_to_plot.sort_values(by=[coluna])

  f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel='Education Level', ylabel='Frequency')
  f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

In [60]:
coluna = 'salario_anual'
titulos = ["Customer's Year Payment", "Compliant Customer's Year Payment", "Non-Compliant Customer's Year Payment"]

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  df_to_plot = dataframe[coluna].value_counts().to_frame()
  df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
  df_to_plot[coluna] = df_to_plot.index
  df_to_plot.reset_index(inplace=True, drop=True)
  df_to_plot.sort_values(by=[coluna], inplace=True)

  f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel='Year_Payment', ylabel='Frequency')
  f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)
  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))
  eixo += 1

figura.show()

In [61]:
df.drop(['id', 'default'], axis=1).select_dtypes('number').head(n=5)

# Transaction count in the last 12 mounths

In [83]:
coluna = 'qtd_transacoes_12m'
titulos = ['Transaction amount last year', 'Compliant customers transactions amount last year', 'Non compliant customers transactions amount last year']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel='Transactions', ylabel='Frequency')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

As we can see, maybe there's some trouble with customers that has between 30 and 60 transactions

In [78]:
coluna = 'valor_transacoes_12m'
titulos = ['Transaction Value Last Year', 'Compliant customers Transaction Value Last Year', 'Valor das Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel='Transaction value', ylabel='Frequency')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

And now we can see this at $2500s

In [82]:
f = sns.relplot(x='valor_transacoes_12m', y='qtd_transacoes_12m', data=df, hue='default')
_ = f.set(
    title='Relation between Value and Amount', 
    xlabel='Value', 
    ylabel='Amount'
  )

As we can see, it could be hard to keep been a compliant customer 
with 40 transactions in $2500

And so in 70 in $8750s avg